In [1]:
%tensorflow_version 1.x
!wget https://www.dropbox.com/s/sakfqp6o8pbgasm/data.tgz
!tar xvzf data.tgz

TensorFlow 1.x selected.
--2020-06-01 08:24:13--  https://www.dropbox.com/s/sakfqp6o8pbgasm/data.tgz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/sakfqp6o8pbgasm/data.tgz [following]
--2020-06-01 08:24:14--  https://www.dropbox.com/s/raw/sakfqp6o8pbgasm/data.tgz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc68d0e425b4bb2ca62a72231a1f.dl.dropboxusercontent.com/cd/0/inline/A41y-z3zsMUgEfwNBz5tZicXx2jCxSd9x8hDXd2I6fGG-CsbQ6vTWh9EKGxze6ynt9kChP3XNLuXQIjk9bhwzoZ0GcMU2x13qcnCDF_r6xsleg/file# [following]
--2020-06-01 08:24:14--  https://uc68d0e425b4bb2ca62a72231a1f.dl.dropboxusercontent.com/cd/0/inline/A41y-z3zsMUgEfwNBz5tZicXx2jCxSd9x8hDXd2I6fGG-CsbQ6vTWh9EKGxze6ynt9kChP3XNLuXQIjk9bhwzoZ0GcMU2x13qcnCDF_r6xsleg/file
Res

In [4]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import numpy as np
from PIL import Image

import keras

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input,decode_predictions

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import ReduceLROnPlateau

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

from keras import backend
from keras.models import Model
from scipy.optimize import fmin_l_bfgs_b
# from scipy.misc import imsave

batch_size = 16
num_classes = 20
epochs = 350



#### LOAD AND TRANSFORM

# Load 
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

# Stats
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

## Transforms
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


x_train /= 255
x_test /= 255


## Labels
y_train=y_train-1

y_test=y_test-1

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

(791, 250, 250, 3)
(791,)
(784, 250, 250, 3)
(784,)


# First experiment


In [5]:

###########################################################


## DEF A BLOCK CONV + BN + GN + CONV + BN + GN + MAXPOOL 
def CBGN(model,filters,lname,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))

    
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))

  model.add(Conv2D(filters, (3, 3), padding='same'))
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2),name=lname))
  
  return model

## DEF CNN TOPOLOGY  1
model1 = Sequential()

model1=CBGN(model1,16,'conv_model1_1',x_train.shape[1:])
model1=CBGN(model1,32,'conv_model1_2')
model1=CBGN(model1,64,'conv_model1_3')
model1.add(Dropout(0.5,name='m1'))

## DEF CNN TOPOLOGY  2
model2 = Sequential()
model2=CBGN(model2,16,'conv_model2_1',x_train.shape[1:])
model2=CBGN(model2,32,'conv_model2_2')
model2=CBGN(model2,64,'conv_model2_3')
model2.add(Dropout(0.5,name='m2'))


def outer_product(x):
  phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])		# Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
  phi_I = tf.reshape(phi_I,[-1,64*64])	        # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
  phi_I = tf.divide(phi_I,31*31)								  # Divide by feature map size [sizexsize]

  y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))		# Take signed square root of phi_I
  z_l2 = tf.nn.l2_normalize(y_ssqrt, dim=1)								              # Apply l2 normalization
  return z_l2



conv1=model1.get_layer('m1').output 
conv2=model2.get_layer('m2').output 

x = Lambda(outer_product, name='outer_product')([conv1,conv2])

predictions=Dense(num_classes, activation='softmax', name='predictions')(x)

model = Model(inputs=[model1.input,model2.input], outputs=predictions)
  
model.summary()

## OPTIM AND COMPILE
opt = Adam(lr=0.001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.005)




datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)



def multiple_data_generator(generator, X,Y,bs):
    genX = generator.flow(X, Y,batch_size=bs)
    while True:
      [Xi,Yi] = genX.next()
      yield [Xi,Xi],Yi



## TRAINING with DA and LRA
history=model.fit_generator(multiple_data_generator(datagen,x_train, y_train,batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=([x_test, x_test], y_test),
                            callbacks=[reduce_lr],
                            verbose=1)

## check how to feed test data for validation 

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_13_input (InputLayer)    (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv2d_19_input (InputLayer)    (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 250, 250, 16) 448         conv2d_13_input[0][0]            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 250, 250, 16) 448         conv2d_19_input[0][0]            
____________________________________________________________________________________________

# Second experiment


In [6]:
epochs = 250

model1=VGG16(weights='imagenet', include_top=False)
model1.summary()


model2=VGG16(weights='imagenet', include_top=False)
model2.summary()

for i, layer in enumerate(model1.layers):
    layer.name = layer.name + 'a'
    layer.trainable = True

for i, layer in enumerate(model2.layers):
    layer.name = layer.name + 'b'
    layer.trainable = True

def outer_product(x):
  phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])		# Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
  phi_I = tf.reshape(phi_I,[-1,512*512])	        # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
  phi_I = tf.divide(phi_I,7*7)								  # Divide by feature map size [sizexsize]

  y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))		# Take signed square root of phi_I
  z_l2 = tf.nn.l2_normalize(y_ssqrt, dim=1)								              # Apply l2 normalization
  return z_l2

conv1=model1.layers[-1].output
conv2=model2.layers[-1].output

x = Lambda(outer_product, name='outer_product')([conv1,conv2])

predictions=Dense(num_classes, activation='softmax', name='predictions')(x)

model = Model(inputs=[model1.input,model2.input], outputs=predictions)
  
model.summary()

## OPTIM AND COMPILE
opt = Adam(lr=0.001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.005)

datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)



def multiple_data_generator(generator, X,Y,bs):
    genX = generator.flow(X, Y,batch_size=bs)
    while True:
      [Xi,Yi] = genX.next()
      yield [Xi,Xi],Yi




## TRAINING with DA and LRA
history=model.fit_generator(multiple_data_generator(datagen,x_train, y_train,batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=([x_test, x_test], y_test),
                            callbacks=[reduce_lr],
                            verbose=1)




58892288/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [7]:
epochs = 200

model1=VGG16(weights='imagenet', include_top=False)
model1.summary()


model2=VGG16(weights='imagenet', include_top=False)
model2.summary()

for i, layer in enumerate(model1.layers):
    layer.name = layer.name + 'a'
    layer.trainable = False

for i, layer in enumerate(model2.layers):
    layer.name = layer.name + 'b'
    layer.trainable = False

def outer_product(x):
  phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])		# Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
  phi_I = tf.reshape(phi_I,[-1,512*512])	        # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
  phi_I = tf.divide(phi_I,7*7)								  # Divide by feature map size [sizexsize]

  y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))		# Take signed square root of phi_I
  z_l2 = tf.nn.l2_normalize(y_ssqrt, dim=1)								              # Apply l2 normalization
  return z_l2

conv1=model1.layers[-1].output
conv2=model2.layers[-1].output

x = Lambda(outer_product, name='outer_product')([conv1,conv2])

predictions=Dense(num_classes, activation='softmax', name='predictions')(x)

model = Model(inputs=[model1.input,model2.input], outputs=predictions)
  
model.summary()

## OPTIM AND COMPILE
opt = Adam(lr=0.001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, cooldown=1,
                              patience=10, min_lr=0.005)


datagen = ImageDataGenerator(
  width_shift_range=0.2,
  height_shift_range=0.2,
  rotation_range=20,
  zoom_range=[1.0,1.2],
  horizontal_flip=True)



def multiple_data_generator(generator, X,Y,bs):
    genX = generator.flow(X, Y,batch_size=bs)
    while True:
      [Xi,Yi] = genX.next()
      yield [Xi,Xi],Yi

## TRAINING with DA and LRA
history=model.fit_generator(multiple_data_generator(datagen,x_train, y_train,batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=([x_test, x_test], y_test),
                            callbacks=[reduce_lr],
                            verbose=1)




Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     